Ce script génère un rapport statistique complet au format PDF pour un joueur de tennis spécifique, en s'appuyant sur la base de données datatennis.csv

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF
import os

/opt/anaconda3/lib/python3.11/site-packages/fpdf/__init__.py:40: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [2]:
def create_pdf(player_name, image_paths, output_name=".pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    #image_paths, liste avec les noms des images
    for i, img in enumerate(image_paths):

        if i % 2 == 0:
            pdf.add_page()
            y_position = 20
        else:
            y_position = 150

        pdf.image(img, x=10, y=y_position, w=180)

    output_file = player_name.replace(" ", "_") + output_name
    pdf.output(output_file)

    for img in image_paths:
        if os.path.exists(img):
            os.remove(img)

In [3]:
df_all = pd.read_csv("datatennis.csv")

# Pour extraire le nom des joueurs
joueurs = sorted(set(df_all['player1_name']).union(df_all['player2_name']))

print("Tapez le Prenom et le Nom du joueur pour générer son rapport, sans accent (exemple : Rafael Nadal)")
player_name = input("Nom du joueur : ")
if player_name not in joueurs:
    print(f"Le joueur '{player_name}' n’est pas présent dans la base de données.")
    exit()
else:
    print(f"Le joueur {player_name} a été trouvé, génération du rapport...")

Tapez le Prenom et le Nom du joueur pour générer son rapport, sans accent (exemple : Rafael Nadal)
Le joueur Roger Federer a été trouvé, génération du rapport...


In [4]:
image_paths = []
filepath = "datatennis.csv"
filename = player_name.replace(" ", "_") + ".csv"
df = pd.read_csv(filepath)
df_player = df[(df['player1_name'] == player_name) | (df['player2_name'] == player_name)].copy()
df_player.to_csv(filename, index=False)
df = pd.read_csv(filename)
mask = df['player2_name'] == player_name

In [5]:
cols_to_swap = []
for col in df.columns:
    if col.startswith("player1_") and col != "player1_wins" :
        cols_to_swap.append(col)

for col1 in cols_to_swap:
    col2 = col1.replace("player1_", "player2_") 
    df.loc[mask, [col1, col2]] = df.loc[mask, [col2, col1]].values  

df.loc[mask, 'player1_wins'] = 1 - df.loc[mask, 'player1_wins']
df.to_csv(player_name.replace(" ", "_") + ".csv", index=False)

In [6]:
# Statistiques
total_matchs = df.shape[0]
nb_victoires = df['player1_wins'].sum()
taux_victoires = round((nb_victoires / total_matchs) * 100, 1)
rank_moyen = round(df['player2_rank'].mean(), 1)
nationalite = df['player1_ioc'].mode()[0]

# On génère l'image sur le PDF
plt.figure(figsize=(6, 4))
plt.axis("off")

text = (
    f"Profil de {player_name}\n\n"
    f"Nationalité : {nationalite}\n"
    f"Nombre total de matchs : {total_matchs}\n"
    f"Nombre de victoires : {nb_victoires}\n"
    f"Taux de victoire : {taux_victoires}%\n"
    f"Classement moyen des adversaires : {rank_moyen}\n"
)

plt.text(0.5, 0.5, text, fontsize=11, ha='center', va='center', wrap=True)
img0 = "bio_joueur.png"
plt.savefig(img0, bbox_inches='tight') 
plt.close()

image_paths.append(img0) #Pour insérer plus tard dans le PDF

In [7]:
# Répartition des matchs par surface
matchs_par_surface = df['surface'].value_counts()

plt.figure(figsize=(6, 5))
plt.bar(matchs_par_surface.index, matchs_par_surface.values, color='lightgreen') 
plt.xlabel("Surface")
plt.ylabel("Nombre de matchs")
plt.title(f"Répartition des matchs de {player_name} par surface")
plt.xticks(rotation=0)
plt.tight_layout()
img1 = "matchs_par_surface.png"
plt.savefig(img1)
plt.close()
image_paths.append(img1)

In [8]:
# Répartition des matchs par année
matchs_par_annee = df['year'].value_counts().sort_index()

plt.figure(figsize=(10, 5))
plt.bar(matchs_par_annee.index.astype(str), matchs_par_annee.values, color='skyblue')
plt.ylabel("Nombre de matchs")
plt.title(f"Répartition des matchs de {player_name} par année")
plt.xticks(rotation=45)
plt.tight_layout()
img2 = "matchs_par_annee.png"
plt.savefig(img2)
plt.close()
image_paths.append(img2)

In [9]:

#Nombre de matchs en Grand Chelem et en master 1000

grand_chelem = df[df['tourney_level'] == 'G']
masters_1000 = df[df['tourney_level'] == 'M']

matchs_gc_par_annee = grand_chelem['year'].value_counts().sort_index()
matchs_masters_par_annee = masters_1000['year'].value_counts().sort_index()

all_years = sorted(set(matchs_gc_par_annee.index).union(matchs_masters_par_annee.index))

# On rajoute 0 pour les années absentes
gc = [matchs_gc_par_annee.get(year, 0) for year in all_years]
m1000 = [matchs_masters_par_annee.get(year, 0) for year in all_years]

# Histogramme
x = range(len(all_years))
bar_width = 0.4

plt.figure(figsize=(12, 6))
plt.bar([i - bar_width/2 for i in x], gc, width=bar_width, label="Grand Chelem", color='cornflowerblue')
plt.bar([i + bar_width/2 for i in x], m1000, width=bar_width, label="Masters 1000", color='salmon')
plt.xlabel("Année")
plt.ylabel("Nombre de matchs")
plt.title(f"Matchs de {player_name} en Grand Chelem vs Masters 1000 par année")
plt.xticks(ticks=x, labels=all_years, rotation=45)
plt.legend()
plt.tight_layout()

img_path = "matchs_gc_vs_masters.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [10]:
#Tour moyen atteint en grand chelem et en master 1000

# Mapping des tours
round_mapping = {
    'R128': 1, 'R64': 2, 'R32': 3, 'R16': 4,
    'QF': 5, 'SF': 6, 'F': 7, 'W': 8
}
round_labels = {
    1: "1er tour", 2: "2e tour", 3: "3e tour",
    4: "Huitièmes", 5: "Quarts", 6: "Demi-finales",
    7: "Finale", 8: "Vainqueur"
}
df['round_num'] = df['round'].map(round_mapping) #On rajoute une colonne avec les "numérotations" de chaque tour

df_gc = df[df['tourney_level'] == 'G']
df_masters = df[df['tourney_level'] == 'M']

# Moyenne du tour max par tournoi puis par année
gc = df_gc.groupby(['year', 'tourney_id'])['round_num'].max().groupby('year').mean()
m1000 = df_masters.groupby(['year', 'tourney_id'])['round_num'].max().groupby('year').mean()

In [11]:
# Histogramme Grand Chelem
plt.figure(figsize=(10, 4))
plt.bar(gc.index.astype(str), gc.values, color='cornflowerblue')
plt.title("Tour moyen atteint en Grand Chelem")
plt.ylabel("Tour")
plt.yticks(ticks=list(round_labels.keys()), labels=list(round_labels.values()))
plt.xticks(rotation=45)
plt.tight_layout()
img1 = "tour_moyen_grand_chelem.png"
plt.savefig(img1)
plt.close()
image_paths.append(img1)

In [12]:
# Histogramme Masters 1000
plt.figure(figsize=(10, 4))
plt.bar(m1000.index.astype(str), m1000.values, color='salmon')
plt.title("Tour moyen atteint en Masters 1000")
plt.ylabel("Tour")
plt.yticks(ticks=list(round_labels.keys()), labels=list(round_labels.values()))
plt.xticks(rotation=45)
plt.tight_layout()
img2 = "tour_moyen_masters1000.png"
plt.savefig(img2)
plt.close()
image_paths.append(img2)

In [13]:
#Temps moyen des matchs sur chaque Grand Chelem et sur les autres tournois

# Liste des 4 tournois du Grand Chelem
gc_tourneys = {
    "Australian Open": "AO",
    "Roland Garros": "RG",
    "Wimbledon": "WIM",
    "US Open": "USO"
}

# Pour chaque Grand Chelem
for nom, code in gc_tourneys.items():
    df_gc = df[(df['tourney_level'] == 'G') & (df['tourney_name'].str.contains(nom))]
    mean_duration = df_gc.groupby('year')['minutes'].mean()

    plt.figure(figsize=(10, 4))
    plt.bar(mean_duration.index.astype(str), mean_duration.values, color='cornflowerblue')
    plt.title(f"Temps moyen des matchs – {nom}")
    plt.ylabel("Minutes")
    plt.xticks(rotation=45)
    plt.tight_layout()

    img_path = f"temps_moyen_{code}.png"
    plt.savefig(img_path)
    plt.close()
    image_paths.append(img_path)

# BO3 (tous les autres tournois)
df_bo3 = df[df['tourney_level'] != 'G']
bo3_mean = df_bo3.groupby('year')['minutes'].mean()

plt.figure(figsize=(10, 4))
plt.bar(bo3_mean.index.astype(str), bo3_mean.values, color='salmon')
plt.title("Temps moyen des matchs – BO3 (hors Grand Chelem)")
plt.ylabel("Minutes")
plt.xticks(rotation=45)
plt.tight_layout()

img_path = "temps_moyen_bo3.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [14]:
# Moyenne globale en Grand Chelem et hors Grand Chelem
bo5 = df[df['tourney_level'] == 'G']
bo3 = df[df['tourney_level'] != 'G']
bo5_avg = bo5['minutes'].mean()
bo3_avg = bo3['minutes'].mean()


plt.figure(figsize=(6, 2))
plt.axis('off')
plt.text(0.5, 0.6, f"Temps moyen en BO5 (GC) : {bo5_avg:.1f} min", fontsize=12, ha='center')
plt.text(0.5, 0.3, f"Temps moyen en BO3 : {bo3_avg:.1f} min", fontsize=12, ha='center')
img_path = "resume_temps_moyen_bo3_bo5.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [15]:
# Moyenne d'aces par match, par année

aces_per_match = df.groupby('year')['player1_ace'].mean()

plt.figure(figsize=(10, 4))
plt.bar(aces_per_match.index.astype(str), aces_per_match.values, color='mediumseagreen')
plt.title("Moyenne d’aces par match par année")
plt.xlabel("Année")
plt.ylabel("Aces par match")
plt.xticks(rotation=45)
plt.tight_layout()
img_path = "aces_par_match_par_annee.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [16]:
# Moyenne de double fautes par match, par année
df_per_year = df.groupby('year')['player1_df'].mean()

plt.figure(figsize=(10, 4))
plt.bar(df_per_year.index.astype(str), df_per_year.values, color='indianred')
plt.title("Moyenne de double fautes par match par année")
plt.xlabel("Année")
plt.ylabel("Double fautes par match")
plt.xticks(rotation=45)
plt.tight_layout()
img_path = "double_fautes_par_match_par_annee.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [17]:
#Pourcentage de première balle

df = df[(df['player1_svpt'] > 0) & (df['player1_1stIn'] > 0)]
df['first_serve_pct'] = df['player1_1stIn'] / df['player1_svpt'] #svpt : nombre total de points
ratios = df.groupby('year')['first_serve_pct'].mean() * 100

plt.figure(figsize=(10, 4))
plt.bar(ratios.index.astype(str), ratios.values, color='steelblue')
plt.title("% de premières balles par année")
plt.xlabel("Année")
plt.ylabel("Pourcentage")
plt.xticks(rotation=45)
plt.ylim(0, 100)
plt.tight_layout()

img_path = "pourcentage_premieres_balles_par_annee.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [18]:
#% de points gagnés sur première balle et sur deuxième balle, sur le même graphe

df['first_serve_win_pct'] = df['player1_1stWon'] / df['player1_1stIn']
#Deuxième partie : nombre de deuxième balle jouée
df['second_serve_win_pct'] = df['player1_2ndWon'] / (df['player1_svpt'] - df['player1_1stIn'])

ratios = df.groupby('year')[['first_serve_win_pct', 'second_serve_win_pct']].mean() * 100

x = range(len(ratios))
bar_width = 0.4

plt.figure(figsize=(10, 4))
plt.bar([i - bar_width/2 for i in x], ratios['first_serve_win_pct'], width=bar_width, label="Sur 1re", color='mediumseagreen')
plt.bar([i + bar_width/2 for i in x], ratios['second_serve_win_pct'], width=bar_width, label="Sur 2e", color='coral')
plt.xticks(ticks=x, labels=ratios.index.astype(str), rotation=45)
plt.title("% de points gagnés sur 1re et 2e balle")
plt.xlabel("Année")
plt.ylabel("Pourcentage")
plt.ylim(0, 100)
plt.legend()
plt.tight_layout()

img_path = "points_gagnes_sur_1re_2e_balle.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [19]:
# % de victoires (moyenne des 1 dans player1_wins)
win_rate_year = df.groupby('year')['player1_wins'].mean() * 100

plt.figure(figsize=(10, 4))
plt.bar(win_rate_year.index.astype(str), win_rate_year.values, color='dodgerblue')
plt.title("Pourcentage de victoires par année")
plt.xlabel("Année")
plt.ylabel("Pourcentage de victoires")
plt.xticks(rotation=45)
plt.ylim(0, 100)
plt.tight_layout()

img_path = "pourcentage_victoires_par_annee.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [20]:
# Calcul du % de victoires par année, surface

win_rate = df.groupby(['year', 'surface'])['player1_wins'].mean().unstack() * 100

colors = {
    'Hard': 'royalblue',
    'Clay': 'firebrick',
    'Grass': 'forestgreen'
}
win_rate.plot(kind='bar', figsize=(12, 5), color=[colors.get(surf, 'gray') for surf in win_rate.columns])
plt.title("Pourcentage de victoires par surface et par année")
plt.xlabel("Année")
plt.ylabel("Pourcentage de victoires")
plt.xticks(rotation=45)
plt.ylim(0, 100)
plt.tight_layout()

img_path = "pourcentage_victoires_surface_annee.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [21]:
# Classement le plus récent de chaque année (dernier match joué)
last_rank = df.groupby('year')['player1_rank'].last()


plt.figure(figsize=(10, 4))
plt.plot(last_rank.index, last_rank.values, marker='o', linestyle='-', color='purple')
plt.gca().invert_yaxis() 
plt.title("Classement ATP (dernier de l’année)")
plt.xlabel("Année")
plt.ylabel("Classement ATP")
plt.xticks(rotation=45)
plt.tight_layout()

img_path = "classement_atp_par_annee.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [22]:
# Total de points le plus récent de chaque année
last_points = df.groupby('year')['player1_rank_points'].last()

plt.figure(figsize=(10, 4))
plt.plot(last_points.index, last_points.values, marker='o', linestyle='-', color='darkorange')
plt.title("Points ATP (dernier total de l’année)")
plt.xlabel("Année")
plt.ylabel("Points ATP")
plt.xticks(rotation=45)
plt.tight_layout()

img_path = "points_atp_par_annee.png"
plt.savefig(img_path)
plt.close()
image_paths.append(img_path)

In [23]:
create_pdf(player_name, image_paths)